In [242]:
# imports and global vars
import numpy as np
import math
import random

input_layer = 3
hidden_layer = 10
output_layer = 3
m = 0.5 
c = 1   # play with values and such

run_time = 6

neurons_input = []
neurons_hidden = []
neurons_output = []

syn_layer_1 = []
syn_layer_2 = []
syn_outputs = []

current_time = 0
time_step = 1
inputs = [1.4, 0.8, 0.9, 1.0] # need to match input #

a = 1 # the main part of what i will be changing for weights changes the size of the sigmoid without changing near zero
b = 0  # bias will add to updating with ^ later
m = 0.5 # the rate of change of the sigmoid 
c = 2 # a constant I used, same as but but shift the sigmod to a place where i 0 is close to 0 (actuyll 0.04 but whatever)
layer_bias = -1 #



In [243]:
# Skuld Function Back Box

def ians_sigmoid(x, a_a, b_b, m_m, c_c, layer=0):
    return (1/ ((1/a_a) + np.exp(((-x*m_m) + b_b + c_c + layer * layer_bias))))* a_a

def calc_loss(output, desired):
    return (0.5 * ((output - desired) ** 2))

def d_loss_d_weight_2(n1, n2, desired):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1) # removedall layer_bias can check layer if correct call

def d_loss_d_weight_1(n1, n2, desired, w2, input):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1 * w2 * (1- n1) * input)

def d_loss_d_bias_2(n2, desired):
    return (n2 - desired) * (n2 - layer_bias) * (1 - (n2 - layer_bias))

def d_loss_d_bias_1(n1, n2, delta2, desired, a2):
    return delta2 * a2 * (n1 - layer_bias) * (1 - (n1 - layer_bias))

def update_param(lr, grad, w):
    w -= lr * grad
    return w

class Neuron: 
    def __init__(self, layer, number):

        self.layer = layer
        self.number = number
    

        self.current_layer = 0
        self.next_layer = 0
        self.past_layer = 1
        self.input_syn = 0
        self.output_syn = 0
        
        
        self.spikes = 0
        #self.firetime = -1
        self.fired = False

        self.set_outputs(layer)

        if self.layer == 0:
            self.threshold = 1.0
        else: 
            self.threshold = 1.0  #2/3 * (self.past_layer)   #+ (self.current_layer + 1) # previous layer # of n /2 seem like a good fit  # will change to fun that reflect # of inputs and layer -- number of inputs
        self.voltage = 0

        #self.collected = 0
        self.scheduled_spike = -10 # change to just runtime or somthng else
        self.spike_time = 10.0
        

    def set_outputs(self, layer):
        '''Called with the creation of neuron create all the correct assocations for later here'''
        global input_layer, output_layer, syn_layer_1, syn_layer_2, hidden_layer, syn_outputs, past_layer
        if layer == 0:
            self.next_layer = hidden_layer
            #self.input_syn = 0 # remains nothing
            self.output_syn = syn_layer_1
        elif layer == 1:
            self.current_layer = hidden_layer
            self.next_layer = output_layer
            self.past_layer = input_layer
            self.input_syn = syn_layer_1
            self.output_syn = syn_layer_2
        elif layer == 2:
            self.current_layer = output_layer
            self.next_layer = output_layer # same amount of outputs
            self.past_layer = hidden_layer
            self.input_syn = syn_layer_2
            self.output_syn = syn_outputs
  

    def send_to_syn(self, time, delay):
        '''Sends out the times to the synapses of when they should arive at their next neuron'''
        global l1_a, l2_a, l1_b, l2_b, m, c
        self.fired = True
        #print(self.layer, self.number, time, delay)
        start = int(self.number * self.next_layer)
        for syn in self.output_syn[start:(start+(self.next_layer))]:
            #print('input: ', syn.input_neuron, "output ", syn.output_neuron, "on layer ", syn.layer)
            # if self.layer == 1:
            #     a = l1_a[syn.input_neuron][syn.output_neuron]
            #     b = l1_b[syn.input_neuron][syn.output_neuron]
            # if self.layer == 2:
            #     a = l2_a[syn.input_neuron][syn.output_neuron]
            #     b = l2_b[syn.input_neuron][syn.output_neuron]
            syn.spike_time = ians_sigmoid(time, syn.a, syn.b, m, c, layer=syn.layer) + delay
            #print(ians_sigmoid(time, 1, 0, m, c, layer=syn.layer) + delay)
            #print(ians_sigmoid(time, 2, 0, m, c, layer=syn.layer) + delay)
            #print(syn.spike_time, syn.layer)
            #print("syn a and b", a, b)
            #print("syna and b", syn.a, syn.b)
            #print("syn spike time & delay ", syn.spike_time, delay)
            #print("spike time send to syn ",syn.input_neuron,syn.output_neuron, "at time ", syn.spike_time)
            pass
        # for all sync going out from this one: 
            # set call ian_sig and set that synapaces time to reach at calculated time


    def check_arival(self):
        '''Check first if this neuron has alreay sent or not. If it hasn't then it goes to check the synapses to see if one has arrived'''
        if self.fired:
            return
        else:
            self.check_syn_arival()


    def check_syn_arival(self):
        '''used in check_arival it will see if neruon is ready to act now basied on arriving spikes'''
        global current_time
        #print(self.layer)
        if self.layer == 0: # only for input layer
            if self.scheduled_spike <= current_time: # if fired already checked
                self.voltage += 1
                #print("input voltage set at time ", current_time)
        else:

            start = self.number * (self.current_layer -1)
            for syn in self.input_syn[start: : self.current_layer]: # goes over all incoming synapaces
                if (syn.message_complete) == False: # if syn is has not yet given this neuron its message
                    #print(self.layer, syn.layer)
                    #print(syn.check_spike(), current_time, syn.layer)
                    # print(syn.check_spike(), current_time, syn.layer)
                    if syn.check_spike() <= current_time: # will check if current step has been reached
                        #print(syn.check_spike(), current_time)
                        #print(self.layer, syn.layer)
                        #print("got Here")
                        #print(syn.check_spike(), current_time, self.layer)
                        # print("layer: ", self.layer)
                        # print("recived spike at: ",current_time)
                        syn.message_complete = True
                        #update neruons voltage
                        self.voltage += 1.2

        self.check_threshold_voltage()
        
                    
                    
    def check_threshold_voltage(self):
        global time_step
        if self.voltage >= self.threshold:
            #print(self.layer, self.voltage, self.threshold)
            #print("self.voltage: ", self.voltage)
            #print("threshold: ", self.threshold)
            #print(current_time)
            self.spike_time = current_time
            #print(self.spike_time)
            #print(self.layer, self.spike_time)
            #print("spike time: ", current_time)
            self.send_to_syn(current_time, current_time)  # CHANGE IF LAYER NOT == 0?
        # else:
        #     if current_time >= (self.layer):
        #         self.voltage += (time_step*self.past_layer) * 0.5 # will change such and whatnot just playing to start# 

class synapse:
    def __init__(self, input_neuron, output_neuron, layer, a, b):
        global run_time
        self.a = a
        self.b = b
        self.input_neuron = input_neuron
        self.output_neuron = output_neuron
        self.layer = layer
        self.spike_time = 10.0
        self.message_complete = False
        self.grad_a = 0
        self.grad_b = 0

    def set_spiketime(self, time):
        self.spike_time = float(time)


    def check_spike(self):
        return float(self.spike_time)

def genesis():
    create_new_neurons_network()
    create_synapses()
    return

def create_new_neurons_network():
    count = 0
    for i in range(len(inputs)):
        n = Neuron(layer=0, number=i)  # will add later to add a and b and such
        #n.set_firetime(val)
        neurons_input.append(n)
        n.scheduled_spike = inputs[i] 
        #print(n.scheduled_spike)
        #print('Neuron Input neuron ' + str(neurons_input[i].number), "scheduled time: ", n.scheduled_spike)

    count = 0
    for i in range (hidden_layer):
        n = Neuron(layer=1, number=i)
        neurons_hidden.append(n)
        #print("Neuron at layer 1: " + str(neurons_hidden[i].number))

    count = 0
    for i in range (output_layer):
        n = Neuron(layer=2, number=i)
        neurons_output.append(n)
        #print("Neuron at output layer: " + str(neurons_output[i].number))

def create_synapses():
    for input_n in neurons_input:
        for hidden_n in neurons_hidden:
            #print(input_n.number)
            #print(hidden_n.number)
            # create new object of synapics 
            #give the object to both neurons so one can output to and the other can edit it
            syn = synapse(input_neuron=input_n.number, output_neuron=hidden_n.number, layer=0, a= 1 + random.uniform(-0.1, 0.1), b= random.uniform(-0.1, 0.1)) 
            syn_layer_1.append(syn)

    for hidden_n in neurons_hidden:
        for output_n in neurons_output:
            #print("h", hidden_n.number)
            #print("0" , output_n.number)
            # create new object of synapics 
            #give the object to both neurons so one can output to and the other can edit it
            syn = synapse(input_neuron=hidden_n.number, output_neuron=output_n.number, layer=1, a= 1 + random.uniform(-0.1, 0.1), b= random.uniform(-0.1, 0.1)) 
            syn_layer_2.append(syn)

    
    for count in range(output_layer): # will check layer 
            syn = synapse(input_neuron=count, output_neuron=count, layer=2, a=a, b=b) 
            syn_outputs.append(syn)


    return 0

def run_simulation(runtime, dx):
    global current_time, time_step
    time_step = 1.0/dx
    
    current_time = 0

    for i in range(runtime*dx):
       
        for n1 in neurons_input:
            n1.check_arival()
            pass 

        for n2 in neurons_hidden:
            n2.check_arival()
            pass 

        for n3 in neurons_output:
            n3.check_arival()
            pass

        current_time += time_step


    return 

def reset_network():
    global current_time
    current_time = 0
    for n in neurons_input: 
        n.voltage = 0
        n.fired = False
        n.spike_time = run_time
    for n in neurons_hidden:
        n.voltage = 0
        n.fired = False
        n.spike_time = run_time
    for n in neurons_output:
        n.voltage = 0
        n.fired = False
        n.spike_time = run_time
    for s in syn_layer_1:
        s.spike_time = run_time
        s.message_complete = False
    for s in syn_layer_2:
        s.spike_time = run_time
        s.message_complete = False

def Skuld_learn(inputs, x,y,z, desired, lr):
    """description here
    """
    global m, c, layer_bias
    
    # genesis()


    for idx, n in enumerate(neurons_input):
        n.scheduled_spike = float(inputs[idx])



    run_simulation(run_time, 100)

    # for i in range(input_layer):
    #     print(neurons_input[i].spike_time)

    avg_input  = sum(n.spike_time for n in neurons_input ) / input_layer
    #print(neurons_input)

    avg_output = 0
    for i in range(output_layer): # goes overall outputs to get the average output
        avg_output += neurons_output[i].spike_time
    avg_output = avg_output/output_layer

    avg_hidden = 0
    for i in range(hidden_layer): # goes overall outputs to get the average output
        avg_hidden += neurons_hidden[i].spike_time
    avg_hidden= avg_hidden/hidden_layer


    # calcs loss
    #loss = calc_loss(avg_output, desired)

        # === Backward pass ===
    # 1) grab each output spike time into a vector
    output_times = np.array([n.spike_time for n in neurons_output])      # shape (z,)
    desired_times = np.array(desired)                                    # shape (z,)

    # 2) compute per‑neuron delta; dividing by z reproduces your old avg logic
    #    so that sum(dL_dO) == (avg_output - avg(desired))
    dL_dO = (output_times - desired_times) / output_layer                # shape (z,)

    for syn in syn_layer_2:
        # 3) pick the delta for the neuron this synapse feeds into:
        delta_o = dL_dO[syn.output_neuron]

        # chain through average (identical to before, since we already divided by z)
        dL_dδ = delta_o  

        # derivative of f wrt a and b
        E    = -m * avg_hidden + syn.b + c + 2 * layer_bias
        expE = np.exp(E)
        u    = (1 / syn.a) + expE

        # ∂f/∂a and ∂f/∂b for f = a / u
        df_da = 1/u + 1/(u * u * syn.a)
        df_db = - syn.a * expE / (u * u)

        syn.grad_a = dL_dδ * df_da
        syn.grad_b = dL_dδ * df_db

        syn.a -= lr * syn.grad_a
        syn.b -= lr * syn.grad_b

    # backprop into the hidden‐layer “average”
    # we sum up all z of those output deltas back into a single scalar
    dL_dh = np.sum(dL_dO)   # this generalizes dL_dh = dL_dO * z

    # hidden layer update remains the same, but now will also scale if you change y
    for syn in syn_layer_1:
        dL_dδ = dL_dh * (1 / hidden_layer)

        E    = -m * avg_input + syn.b + c + 1 * layer_bias
        expE = np.exp(E)
        u    = (1 / syn.a) + expE

        df_da = 1/u + 1/(u * u * syn.a)
        df_db = - syn.a * expE / (u * u)

        syn.grad_a = dL_dδ * df_da
        syn.grad_b = dL_dδ * df_db

        syn.a -= lr * syn.grad_a
        syn.b -= lr * syn.grad_b
    
    return 



In [247]:
# # testing here

inputs = [0.5] * 4 # need to match input #     # # need to investiat my it slows down so mcuh
genesis()
for i in range(15):
    reset_network()
    print(Skuld_learn(inputs, input_layer, hidden_layer, output_layer, 5.0, 0.1))
    

IndexError: list index out of range

In [245]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

input_layer  = 4    
hidden_layer = 10
output_layer = 3
lr = 0.005
#inputs = [0.5, 0.5, 0.9, 1.2]

# 1) Load Iris and normalize features to [0,1]
iris = datasets.load_iris()
X = iris.data
y = iris.target
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X = X * (1.5 - 0.5) + 0.5  # set x to range of 0.5 - 1.5
#print (X)

X_times = np.round(X, 3) # round inputs to 3 decimal places
#print(X_times)

# 1) define your times
correct_time   = 5.0
incorrect_time = 3.0
num_outputs    = 3    # Iris has 3 classes

# 2) build a (n_samples, 3) array filled with incorrect_time
Y_times = np.full((len(y), num_outputs), incorrect_time)

# 3) for each sample i, set the true‐class column to correct_time
Y_times[np.arange(len(y)), y] = correct_time

#print(Y_times[:5])

X_train, X_test, y_train, y_test = train_test_split(
    X_times, Y_times,
    test_size=0.2, random_state=42, stratify=y
)

#print(y_train, y_test)
inputs = X_train[0].tolist()
# print(X_train[0].tolist())

genesis()
reset_network()

# # 6) TRAIN: multiple epochs over all training examples
epochs = 50
for epoch in range(1, epochs+1):
    print(f"Epoch {epoch}/{epochs}")
    for xi, yi in zip(X_train, y_train):
        #print[inputs.shape]

        #print(inputs.shape)
        #print(inputs)
        #print(yi)
        reset_network()
        output = Skuld_learn(
            inputs         =  inputs,
            x         = input_layer,
            y     = hidden_layer,
            z        = output_layer,
            lr             = lr,
            desired   = [1.6, 1.5, 1.5]
        )
        
        #print(output)
    print(neurons_output[0].spike_time, neurons_output[1].spike_time, neurons_output[2].spike_time)
# # 7) TEST on hold‑out set
# print("\n=== Iris SNN Test ===")
# correct = 0
# for xi, yi_label in zip(X_test, y_test):
#     reset_network()
#     out_times = Skuld_learn(
#         inputs         = xi,
#         x         = input_layer,
#         y     = hidden_layer,
#         z        = output_layer,
#         lr             = lr,
#         desired   = yi
#     )
#     # ALWAYS treat out_times as an array of spike‐times:
#     pred = int(np.argmin(out_times))
#     correct += (pred == yi_label)
# acc = correct / len(X_test)
# print(f"         test accuracy = {acc * 100:.2f}%\n")

# # … [final test run] …
# print("=== Final Test Run ===")
# correct = 0
# for xi, yi_label in zip(X_test, y_test):
#     reset_network()
#     out_times = Skuld_learn(
#         inputs         = xi,
#         x         = input_layer,
#         y     = hidden_layer,
#         z        = output_layer,
#         lr             = 0.0,
#         desired   = yi
#     )
#     pred = int(np.argmin(out_times))
#     correct += (pred == yi_label)
#     print(f"in={np.round(xi,2)} → out={np.round(out_times,2)} pred={pred}, true={yi_label}")

# final_acc = correct / len(X_test)
# print(f"\nFinal Accuracy: {final_acc * 100:.2f}%")


Epoch 1/50


IndexError: list index out of range

Only commented out Scratch below here

In [ ]:
# def run_new_111(desired, lr):   # will need to check work on bias/b and make sure everything is correct in this

#     #desired = desired -.032
#     run_simulation(run_time, 1000)
    
#     n1 = neurons_hidden[0].spike_time
#     n2 = neurons_output[0].spike_time
#     #print("n1, n2 : ", n1, n2)
#     #loss = calc_loss(output, desired)

#     delta_a2 = - d_loss_d_weight_2(n1, n2, desired)

#     syn_a1 = syn_layer_1[0].a
#     syn_a2 = syn_layer_2[0].a

#     delta_a1 = - d_loss_d_weight_1(n1, n2, desired, syn_a2, inputs[0]) # trying to just input a as a the weight directly

#     new_a1 = update_param(lr, delta_a1, syn_a1)
#     new_a2 = update_param(lr, delta_a2, syn_a2)

#     syn_b1 = syn_layer_1[0].b
#     syn_b2 = syn_layer_2[0].b

#     delta_b2 = d_loss_d_bias_2(n2, desired) # b is not - but left as + here
#     delta_b1 = d_loss_d_bias_1(n1, n2, delta_b2, desired, syn_b2) # trying to just input a as a the weight directly


#     new_b1 = update_param(lr, delta_b1, syn_b1)
#     new_b2 = update_param(lr, delta_b2, syn_b2)

#     return new_a1, new_a2, new_b1, new_b2, n1, n2


# #run_new_111(input, 1.5, 0.1)



# genesis()

# for i in range(100):
   
#     reset_network() # needs to add reset for all that isn't the genesis call
   
#     sa1, sa2, sb1, sb2, n1, n2 = run_new_111(desired=0.5, lr=.1) # basic range should work just need to find the range and such but can do later then 
#                                 # desired has to be between 1.532 and 3.335
#                                 # also need to fully reset when values overeach to get that 1.532 and 3.335
#                                 # limited ranged based on limit else it crashes and resets to at near start spike time
#                                 # need roundeding else it falls off for speed
#     # l1_a[0][0] = sa1
#     # l2_a[0][0] = sa2
#     # l1_b[0][0] = sb1
#     # l2_b[0][0] = sb2
#     syn_layer_1[0].a = np.round(sa1, 5)
#     syn_layer_2[0].a = np.round(sa2, 5)
#     syn_layer_1[0].b = np.round(sb1, 5)
#     syn_layer_2[0].b = np.round(sb2, 5)

#     print("a1 and a2: ", sa1, sa2)
#     print("b1 and b2: ", sb1, sb2)
#     print(np.round(n2, 3))

# print("a1 and a2: ", sa1, sa2)
# print("b1 and b2: ", sb1, sb2)

# # print(l1_a)
# # print(l1_b)
# # print(l2_a)
# # print(l2_b)


In [ ]:
# create_synapses()      # create synapses objects once
# for epoch in range(10):
#    # clear out last run’s flags & times
#     sa1, sa2, sb1, sb2, n1, n2 = run_new_111(1.5, 0.1)
#     # write back into synapse objects:
#     syn_layer_1[0].a = sa1
#     syn_layer_2[0].a = sa2
#     syn_layer_1[0].b = sb1
#     syn_layer_2[0].b = sb2
#     print(f"Epoch {epoch}: a1={sa1:.4f}, a2={sa2:.4f}, n2={n2:.3f}")


In [ ]:
# import matplotlib.pyplot as plt

# def plot_spike_raster(inputs,
#                       syn_layer_1,
#                       syn_layer_2,
#                       syn_outputs):
#     """
#     inputs: list of input‐neuron spike times
#     syn_layer_1: list of synapse objects for layer 1 (must have .spike_time)
#     syn_layer_2: list of synapse objects for layer 2
#     syn_outputs: list of synapse objects for output layer
#     """
#     # collect times & y‐positions
#     times = []
#     rows  = []
#     labels = []

#     # row 0 = input spikes
#     for t in inputs:
#         times.append(t)
#         rows.append(0)
#         labels.append("input")

#     # row 1 = layer‐1 synapses
#     for idx, s in enumerate(syn_layer_1):
#         times.append(s.spike_time)
#         rows.append(1)
#         labels.append(f"syn1_{idx}")

#     # row 2 = layer‐2 synapses
#     for idx, s in enumerate(syn_layer_2):
#         times.append(s.spike_time)
#         rows.append(2)
#         labels.append(f"syn2_{idx}")

#     # row 3 = output synapses
#     for idx, s in enumerate(syn_outputs):
#         times.append(s.spike_time)
#         rows.append(3)
#         labels.append(f"out_{idx}")

#     # now plot
#     plt.figure(figsize=(8, 3))
#     plt.scatter(times, rows, marker='|', s=200)
#     plt.yticks([0,1,2,3], ['input','syn1','syn2','output'])
#     plt.xlabel("Time")
#     plt.title("Spike / Synapse firing times")
#     plt.tight_layout()
#     plt.show()


# # Example usage:
# # (assuming you kept your syn_layer_1, syn_layer_2, syn_outputs in scope)
# final_time = run_simulation(inputs, run_time, 10000)
# print("final spike at", final_time)

# plot_spike_raster(
#     inputs,
#     syn_layer_1,
#     syn_layer_2,
#     syn_outputs
# )


In [ ]:
# # train
 
# input = 0.5  # all inputs have to be larger then 0.5?
# lr = 0.1
# a1 = 1.1
# a2 = 0.9
# b1 = -0.1
# b2 = 0.0
# desired = 1.0
# # weird thing with moving the lr too much need to keep low nery steep start with changing lower m maybe?
# for i in range(1000):
#     a1, a2, b1, b2, r = run_basic(input, a1, a2, b1, b2, desired, lr)
#     # print("a1 and a2: ", a1, a2)
#     # print("b1 and b2: ", b1, b2)
#     print(np.round(r[0][1], 3))
# print("a1 and a2: ", a1, a2)
# print("b1 and b2: ", b1, b2)
# print(np.round(r[0][1], 3))





In [ ]:
# # Brian Simulation: 
# from brian2 import *
# import numpy as np
# import logging
# import warnings
# from brian2 import prefs, set_device
# # Tell Brian2 to use the Cython code generator:
# prefs.codegen.target = 'cython'

# # Optionally compile but keep Python interface:
# set_device('runtime')  # default; compiles operations to .so but stays in Python process

# # suppress overflow warnings
# warnings.filterwarnings('ignore', category=RuntimeWarning)
# numpy.seterr(over='ignore', under='ignore')
# logging.getLogger('brian2').setLevel(logging.ERROR)

# # ----------------------------------------------------------------------------
# # Spike timing and derivative
# layer_bias = 0.5
# start_scope()
# defaultclock.dt = 0.001*ms

# @implementation('numpy', discard_units=True)
# @check_units(time=1, a=1, b=1, m=1, c=1, layer=1, result=1)
# def ians_sigmoid(time, a, b, m, c, layer):
#     print(time)
#     global layer_bias
#     print("result of ians_sig: ", 1/ ((1/a) + np.exp(((-time*m) + b + c + layer * layer_bias))))
#     return 1/ ((1/a) + np.exp(((-time*m) + b + c + layer * layer_bias)))



# def mini1x1x1(inputs):
#     """
#     Mini 1x1x1 network with a single input, hidden, and output neuron.
#     This is a minimal example to demonstrate the basic structure of a spiking neural network.
#     """
#     # Define network sizes
#     n_input = 1
#     n_hidden = 1
#     n_output = 1
#     n_total = n_input + n_hidden + n_output

#     # Full neuron group
#     neurons = NeuronGroup(n_total, '''
#         v : 1
#         sum : 1
#         spikes_received : 1
#         scheduled_time : second
#         global_clock : 1
#         spiked : boolean
#     ''', threshold='v > 0.5', reset='''
#     v = 0
#     ''', method='exact')
#     neurons.v = 0
#     neurons.scheduled_time = 1e9 * second
#     neurons.global_clock = 0.0
#     neurons.sum = 0.0
#     neurons.spikes_received = 0.0

#     # Spike inputs (one per input neuron)     
#     stim = SpikeGeneratorGroup(n_input, indices=range(n_input), times=(inputs) * ms)

#     # Input → Hidden connections
#     syn_input = Synapses(stim, neurons[0:n_input], '''
#         layer : 1
#         a : 1
#         b : 1
#     ''', on_pre='''
#         spikes_received += 1
#         scheduled_time = (ians_sigmoid(t/ms, a, b, 3, 5, 0) + t/ms) * ms 
#     ''') 
#     syn_input.connect(j='i')  # connect stim[i] to neurons[i]
#     syn_input.layer = 0
#     syn_input.a = 5.0
#     syn_input.b = 0.0


#     # Hidden layer: input → hidden
#     syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input + n_hidden], '''
#         w : 1
#         layer : 1
#     ''', on_pre='''
#         spikes_received += 1
#         scheduled_time = (ians_sigmoid(t/ms, a, b, 3, 5, 0) + t/ms) * ms 
#     ''')
#     syn_hidden.connect()
#     #syn_hidden.w = w1 
#     syn_hidden.layer = 1

#     # Output layer: hidden → output
#     # syn_output = Synapses(neurons[n_input:n_input + n_hidden], neurons[n_input + n_hidden:n_total], '''
#     #     w : 1
#     #     layer : 1
#     # ''', on_pre='''
#     #     spikes_received += 1
#     #     v += 1
    
#     # ''')
#     # syn_output.connect()
#     # #syn_output.w = w2 
#     # syn_output.layer = 2

#     window = 0.02*ms

#     neurons.run_regularly('''
                          
#         v = int(abs(t - scheduled_time) < 0.005*ms) * 1.2
#         in_window = (scheduled_time >= (t - 0.02*ms)) * (scheduled_time <= (t + 0.02*ms))
   
#         fire_flag = int(in_window * (1 - spiked))
        
#         v      = fire_flag * 2
#         spiked = spiked or (fire_flag > 0)
  
#     ''', dt=0.001*ms)

#         #v = int(abs(t - scheduled_time) < 0.005*ms) * 1.2

#     # Monitors
#     mon = StateMonitor(neurons, 'v', record=True, dt=0.0001*ms)
#     mon_sum = StateMonitor(neurons, 'sum', record=True)
#     sp_mon = StateMonitor(neurons, 'spikes_received', record=True)
#     sch_time = StateMonitor(neurons, 'scheduled_time', record=True)


#     spikemon = SpikeMonitor(neurons)


#     run(5*ms)

#     # Plot voltages
#     figure(figsize=(10, 6))
#     for i in range(n_total):  # All neurons
#         plot(mon.t/ms, mon.v[i], label=f'Neuron {i}')
#     xlabel('Time (ms)')
#     ylabel('Membrane potential')
#     legend()
#     title('SNN Spike Propagation Across All Layers')
#     show()

#     # plot(mon_sum.t/ms, mon_sum.sum[3])  # or any neuron index
#     # print(mon_sum.sum[1])
#     # print(sp_mon.spikes_received[1])
#     # print(sch_time.scheduled_time[1])

#     for i in range(n_total):
#         times = spikemon.spike_trains()[i]
#         if len(times) > 0:
#             formatted_times = [f"{t/ms:.3f} ms" for t in times]
#             print(f"Neuron {i} spike times: {formatted_times}")

# input = [0.5]

# mini1x1x1(input)
